# Playing with Linear Algebra

Julia Matrix Operation is fast. To know how fast it is, you need a benchmark package. Type `]` and `add BenchmarkTools` in your REPL.

In [5]:
using BenchmarkTools

a = randn(ComplexF64, 4, 4)
b = randn(ComplexF64, 4)
@benchmark $a*$b

BenchmarkTools.Trial: 
  memory estimate:  144 bytes
  allocs estimate:  1
  --------------
  minimum time:     99.982 ns (0.00% GC)
  median time:      110.144 ns (0.00% GC)
  mean time:        128.364 ns (9.09% GC)
  maximum time:     56.150 μs (99.68% GC)
  --------------
  samples:          10000
  evals/sample:     911

### Cool, can we be faster? Yes if the matrix is small!

For static array, we can avoid all allocations.

##### Challenge!
Read the document of this package
https://github.com/JuliaArrays/StaticArrays.jl
and show it is really fast!

In [4]:
# ] add StaticArrays
using StaticArrays

sa = SMatrix{4, 4}(a)
sb = SVector{4}(b)
@benchmark $sa*$sb

@test 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     11.442 ns (0.00% GC)
  median time:      11.732 ns (0.00% GC)
  mean time:        12.272 ns (0.00% GC)
  maximum time:     53.112 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

# If matrix contraction can not satisfy you, tensor contraction will

In [15]:
# ] add TensorOperations
using TensorOperations

ta = randn(6, 10, 5)
tb = randn(6, 5, 7)

@tensor tc[l,j] := ta[i,j,k] * tb[i,k,l]

7×10 Array{Float64,2}:
  3.88092      4.22942  -2.20287   …   4.48744   -12.2507   -1.03379 
  3.69114     -6.52209  -6.9612        1.74549    -5.75151  -1.31186 
  1.98873      3.22936   0.537409     -2.63146    -7.1749    0.463595
 -2.06479    -11.1186   -2.61311       1.88026    -6.39603   2.12045 
 -0.0402845   -2.51435  -0.507906     -1.8457     -5.47197   5.49651 
 -2.76435      4.70284  -1.07937   …   0.342825   -6.35544  -1.00031 
 -1.1936       6.40223   0.103184     -2.27151     3.78841  -7.63995 

# Cool, see what kind of linear algebras we can do

In [8]:
a = randn(100, 100)

using LinearAlgebra
# svd decomposition
svd(a)
# eigen solver
eigen(a)
# qr decomposition
qr(a);

# Large sparse matrix

In [16]:
using SparseArrays
sp = sprand(1000, 1000, 0.01)

# ] add Arpack
using Arpack
eigs(sp, which="SA")

┌ Info: Precompiling Arpack [7d9fca2a-8960-54d3-9f78-7d1dccf2cb97]
└ @ Base loading.jl:1186
ERROR: LoadError: No deps.jl file could be found. Please try running Pkg.build("Arpack").
Currently, the build command might fail when Julia has been built from source
and the recommendation is to use the official binaries from julialang.org.
For more info see https://github.com/JuliaLinearAlgebra/Arpack.jl/issues/5.

Stacktrace:
 [1] top-level scope at /home/leo/.julia/packages/Arpack/WP3ru/src/Arpack.jl:19
 [2] include at ./boot.jl:317 [inlined]
 [3] include_relative(::Module, ::String) at ./loading.jl:1038
 [4] include(::Module, ::String) at ./sysimg.jl:29
 [5] top-level scope at none:2
 [6] eval at ./boot.jl:319 [inlined]
 [7] eval(::Expr) at ./client.jl:389
 [8] top-level scope at ./none:3
in expression starting at /home/leo/.julia/packages/Arpack/WP3ru/src/Arpack.jl:16


ErrorException: Failed to precompile Arpack [7d9fca2a-8960-54d3-9f78-7d1dccf2cb97] to /home/leo/.julia/compiled/v1.0/Arpack/X5VZL.ji.

In [42]:
using KrylovKit

sp = sp'+sp
vals, vecs, info = eigsolve(sp, 1, :SR);

using Test
@test vals[1] ≈ minimum(eigen(sp |> Matrix).values)

Test Passed

In [39]:
?eigen

search: eigen eigen! Eigen rayleighextension rayleighquotient GeneralizedEigen



```
eigen(A; permute::Bool=true, scale::Bool=true) -> Eigen
```

Computes the eigenvalue decomposition of `A`, returning an `Eigen` factorization object `F` which contains the eigenvalues in `F.values` and the eigenvectors in the columns of the matrix `F.vectors`. (The `k`th eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

The following functions are available for `Eigen` objects: [`inv`](@ref), [`det`](@ref), and [`isposdef`](@ref).

For general nonsymmetric matrices it is possible to specify how the matrix is balanced before the eigenvector calculation. The option `permute=true` permutes the matrix to become closer to upper triangular, and `scale=true` scales the matrix by its diagonal elements to make rows and columns more equal in norm. The default is `true` for both options.

# Examples

```jldoctest
julia> F = eigen([1.0 0.0 0.0; 0.0 3.0 0.0; 0.0 0.0 18.0])
Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
  1.0
  3.0
 18.0
eigenvectors:
3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

julia> F.values
3-element Array{Float64,1}:
  1.0
  3.0
 18.0

julia> F.vectors
3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

julia> vals, vecs = F; # destructuring via iteration

julia> vals == F.values && vecs == F.vectors
true
```

---

```
eigen(A, B) -> GeneralizedEigen
```

Computes the generalized eigenvalue decomposition of `A` and `B`, returning a `GeneralizedEigen` factorization object `F` which contains the generalized eigenvalues in `F.values` and the generalized eigenvectors in the columns of the matrix `F.vectors`. (The `k`th generalized eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

# Examples

```jldoctest
julia> A = [1 0; 0 -1]
2×2 Array{Int64,2}:
 1   0
 0  -1

julia> B = [0 1; 1 0]
2×2 Array{Int64,2}:
 0  1
 1  0

julia> F = eigen(A, B);

julia> F.values
2-element Array{Complex{Float64},1}:
 0.0 + 1.0im
 0.0 - 1.0im

julia> F.vectors
2×2 Array{Complex{Float64},2}:
  0.0-1.0im   0.0+1.0im
 -1.0-0.0im  -1.0+0.0im

julia> vals, vecs = F; # destructuring via iteration

julia> vals == F.values && vecs == F.vectors
true
```

---

```
eigen(A::Union{SymTridiagonal, Hermitian, Symmetric}, irange::UnitRange) -> Eigen
```

Computes the eigenvalue decomposition of `A`, returning an `Eigen` factorization object `F` which contains the eigenvalues in `F.values` and the eigenvectors in the columns of the matrix `F.vectors`. (The `k`th eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

The following functions are available for `Eigen` objects: [`inv`](@ref), [`det`](@ref), and [`isposdef`](@ref).

The `UnitRange` `irange` specifies indices of the sorted eigenvalues to search for.

!!! note
    If `irange` is not `1:n`, where `n` is the dimension of `A`, then the returned factorization will be a *truncated* factorization.


---

```
eigen(A::Union{SymTridiagonal, Hermitian, Symmetric}, vl::Real, vu::Real) -> Eigen
```

Computes the eigenvalue decomposition of `A`, returning an `Eigen` factorization object `F` which contains the eigenvalues in `F.values` and the eigenvectors in the columns of the matrix `F.vectors`. (The `k`th eigenvector can be obtained from the slice `F.vectors[:, k]`.)

Iterating the decomposition produces the components `F.values` and `F.vectors`.

The following functions are available for `Eigen` objects: [`inv`](@ref), [`det`](@ref), and [`isposdef`](@ref).

`vl` is the lower bound of the window of eigenvalues to search for, and `vu` is the upper bound.

!!! note
    If [`vl`, `vu`] does not contain all eigenvalues of `A`, then the returned factorization will be a *truncated* factorization.



In [21]:
?eigsolve

search: eigsolve



```
eigsolve(A::AbstractMatrix, [howmany = 1, which = :LM, T = eltype(A)]; kwargs...)
eigsolve(f, n::Int, [howmany = 1, which = :LM, T = Float64]; kwargs...)
eigsolve(f, x₀, [howmany = 1, which = :LM]; kwargs...)
eigsolve(f, x₀, howmany, which, algorithm)
```

Compute `howmany` eigenvalues from the linear map encoded in the matrix `A` or by the function `f`. Return eigenvalues, eigenvectors and a `ConvergenceInfo` structure.

### Arguments:

The linear map can be an `AbstractMatrix` (dense or sparse) or a general function or callable object. If an `AbstractMatrix` is used, a starting vector `x₀` does not need to be provided, it is then chosen as `rand(T, size(A,1))`. If the linear map is encoded more generally as a a callable function or method, the best approach is to provide an explicit starting guess `x₀`. Note that `x₀` does not need to be of type `AbstractVector`, any type that behaves as a vector and supports the required methods (see KrylovKit docs) is accepted. If instead of `x₀` an integer `n` is specified, it is assumed that `x₀` is a regular vector and it is initialized to `rand(T,n)`, where the default value of `T` is `Float64`, unless specified differently.

The next arguments are optional, but should typically be specified. `howmany` specifies how many eigenvalues should be computed; `which` specifies which eigenvalues should be targetted. Valid specifications of `which` are given by

  * `LM`: eigenvalues of largest magnitude
  * `LR`: eigenvalues with largest (most positive) real part
  * `SR`: eigenvalues with smallest (most negative) real part
  * `LI`: eigenvalues with largest (most positive) imaginary part, only if `T <: Complex`
  * `SI`: eigenvalues with smallest (most negative) imaginary part, only if `T <: Complex`
  * [`ClosestTo(λ)`](@ref): eigenvalues closest to some number `λ`

!!! note "Note about selecting `which` eigenvalues"
    Krylov methods work well for extremal eigenvalues, i.e. eigenvalues on the periphery of the spectrum of the linear map. Even with `ClosestTo`, no shift and invert is performed. This is useful if, e.g., you know the spectrum to be within the unit circle in the complex plane, and want to target the eigenvalues closest to the value `λ = 1`.


The argument `T` acts as a hint in which `Number` type the computation should be performed, but is not restrictive. If the linear map automatically produces complex values, complex arithmetic will be used even though `T<:Real` was specified.

### Return values:

The return value is always of the form `vals, vecs, info = eigsolve(...)` with

  * `vals`: a `Vector` containing the eigenvalues, of length at least `howmany`, but could be   longer if more eigenvalues were converged at the same cost. Eigenvalues will be real if   [`Lanczos`](@ref) was used and complex if [`Arnoldi`](@ref) was used (see below).
  * `vecs`: a `Vector` of corresponding eigenvectors, of the same length as `vals`. Note that   eigenvectors are not returned as a matrix, as the linear map could act on any custom Julia   type with vector like behavior, i.e. the elements of the list `vecs` are objects that are   typically similar to the starting guess `x₀`, up to a possibly different `eltype`. In particular,   for a general matrix (i.e. with `Arnoldi`) the eigenvectors are generally complex and are   therefore always returned in a complex number format.   When the linear map is a simple `AbstractMatrix`, `vecs` will be `Vector{Vector{<:Number}}`.
  * `info`: an object of type [`ConvergenceInfo`], which has the following fields

      * `info.converged::Int`: indicates how many eigenvalues and eigenvectors were actually   converged to the specified tolerance `tol` (see below under keyword arguments)
      * `info.residual::Vector`: a list of the same length as `vals` containing the residuals   `info.residual[i] = f(vecs[i]) - vals[i] * vecs[i]`
      * `info.normres::Vector{<:Real}`: list of the same length as `vals` containing the norm   of the residual `info.normres[i] = norm(info.residual[i])`
      * `info.numops::Int`: number of times the linear map was applied, i.e. number of times   `f` was called, or a vector was multiplied with `A`
      * `info.numiter::Int`: number of times the Krylov subspace was restarted (see below)

!!! warning "Check for convergence"
    No warning is printed if not all requested eigenvalues were converged, so always check if `info.converged >= howmany`.


### Keyword arguments:

Keyword arguments and their default values are given by:

  * `krylovdim = 30`: the maximum dimension of the Krylov subspace that will be constructed.   Note that the dimension of the vector space is not known or checked, e.g. `x₀` should not   necessarily support the `Base.length` function. If you know the actual problem dimension   is smaller than the default value, it is useful to reduce the value of `krylovdim`, though   in principle this should be detected.
  * `tol = 1e-12`: the requested accuracy (corresponding to the 2-norm of the residual for   Schur vectors, not the eigenvectors). If you work in e.g. single precision (`Float32`),   you should definitely change the default value.
  * `maxiter = 100`: the number of times the Krylov subspace can be rebuilt; see below for   further details on the algorithms.
  * `issymmetric`: if the linear map is symmetric, only meaningful if `T<:Real`
  * `ishermitian`: if the linear map is hermitian

The default value for the last two depends on the method. If an `AbstractMatrix` is used, `issymmetric` and `ishermitian` are checked for that matrix, ortherwise the default values are `issymmetric = false` and `ishermitian = T <: Real && issymmetric`.

### Algorithm

The last method, without default values and keyword arguments, is the one that is finally called, and can also be used directly. Here, one specifies the algorithm explicitly as either [`Lanczos`](@ref), for real symmetric or complex hermitian problems, or [`Arnoldi`](@ref), for general problems. Note that these names refer to the process for building the Krylov subspace, but the actual algorithm is an implementation of the Krylov-Schur algorithm, which can dynamically shrink and grow the Krylov subspace, i.e. the restarts are so-called thick restarts where a part of the current Krylov subspace is kept.

!!! note "Note about convergence"
    In case of a general problem, where the `Arnoldi` method is used, convergence of an eigenvalue is not based on the norm of the residual `norm(f(vecs[i]) - vals[i]*vecs[i])` for the eigenvectors but rather on the norm of the residual for the corresponding Schur vectors.

    See also [`schursolve`](@ref) if you want to use the partial Schur decomposition directly, or if you are not interested in computing the eigenvectors, and want to work in real arithmetic all the way true (if the linear map and starting guess are real).



In [75]:
using DelimitedFiles

a = randn(ComplexF64, 3,3)
open("test.dat", "w") do f
    writedlm(f, a)
end

In [80]:
open("test.dat", "r") do f
    b = readdlm(f)
end
b

4-element Array{Complex{Float64},1}:
  1.2601337618884032 - 0.02995058821260156im
 -0.4040283034459509 - 0.24804825719381837im
 0.07503132729439094 - 0.0833673573023801im 
 -0.2387132866942657 - 0.46710589398181934im

# Random Numbers?

# Now, you can embrace the world of Notebooks!
# But not programmer's world
# This is why we have the next session

# Defining functions

In [56]:
f(x) = x^2
function f(x::Vector{T}) where T
    append!(copy(x), x)
end
@test f(3) == 9

x = [1,2,3,4]
f.(x)   # now we must add a `.`

4-element Array{Int64,1}:
  1
  4
  9
 16

In [60]:
function f(x::Vector{T}) where T
    append!(copy(x), x)
end

x = [1,2,3,4]
println(f(x))  # calling the vector version function
println(x)

[1, 2, 3, 4, 1, 2, 3, 4]
[1, 2, 3, 4]


In [59]:
# inplace vector version of f
function f!(x::Vector{T}) where T
    append!(x, x)
end

x = [1,2,3,4]
println(f!(x))
x |> println

[1, 2, 3, 4, 1, 2, 3, 4]
[1, 2, 3, 4, 1, 2, 3, 4]


In [64]:
f(x::String, y::String) = x * " " * y
f(x::String) = x^2

x = ["I", "love", "China!"]
reduce(f, x) |> println
mapreduce(f, f, x) |> println

I love China!
II lovelove China!China!


# Wrap up
* multiple dispatch -> function name can be reused

In [ ]:
# But Why do we prefer multiple-dispatch?